### The topics that we will cover in the following sections include the following:

. Cleaning and preparing text data

. Building feature vectors from text documents

. Training a machine learning model to classify positive and negative movie 
reviews

. Working with large text datasets using out-of-core learning

. Inferring topics from document collections for categorization

#### Preparing the IMDb movie review data for text processing

####            Large Movie Review Dataset

This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. See the README file contained in the release for more details.


http://ai.stanford.edu/~amaas/data/sentiment/

In [1]:
import tarfile

with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()

Preprocessing the movie dataset into more
convenient format

we will use the

Python Progress Indicator 
(PyPrind, https://pypi.python.org/pypi/PyPrind/)
package that I developed several years ago for such purposes. PyPrind can be
installed by executing the pip install pyprind command.

In [5]:
# pip install PyPrind

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyprind
import pandas as pd
import os

# change the 'basepath' to the directory of the 
# unzipped movie dataset

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)

df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file),
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df._append([[txt, labels[l]]], ignore_index=True)
            
            pbar.update()

df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:24:40


In [5]:
import numpy as np

np.random.seed(0)

df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


### Transforming words into feature vectors

To construct a bag-of-words model based on the word counts in the respective
documents, we can use the CountVectorizer class implemented in scikit-learn. As
we will see in the following code section, CountVectorizer takes an array of text
data, which can be documents or sentences, and constructs the bag-of-words model
for us:

In [13]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'])
bag = count.fit_transform(docs)


In [14]:
print(count.vocabulary_)

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


In [15]:
print(bag.toarray())

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


### Assessing word relevancy via term frequency-inverse document frequency (TF-IDF)

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.56 0.56 0.   0.43 0.  ]
 [0.   0.43 0.   0.   0.56 0.43 0.56]
 [0.4  0.48 0.31 0.31 0.31 0.48 0.31]]


In [18]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In the previous subsections, we learned about the bag-of-words model, term bagof-
words􀀃􀁐􀁒􀁇􀁈􀁏􀀏􀀃􀁗􀁈􀁕􀁐􀀃􀁉􀁕􀁈􀁔􀁘􀁈􀁑􀁆􀁌􀁈􀁖􀀏􀀃􀁄􀁑􀁇􀀃􀁗􀁉􀀐􀁌􀁇􀁉􀁖􀀑􀀃􀀫􀁒􀁚􀁈􀁙􀁈􀁕􀀏􀀃􀁗􀁋􀁈􀀃􀃀􀁕􀁖􀁗􀀃􀁌􀁐􀁓􀁒􀁕􀁗􀁄􀁑􀁗􀀃􀁖􀁗􀁈􀁓􀂳
before we build our bag-of-words model—is to clean the text data by stripping it of
all unwanted characters. To illustrate why this is important, let's display the last 50
􀁆􀁋􀁄􀁕􀁄􀁆􀁗􀁈􀁕􀁖􀀃􀁉􀁕􀁒􀁐􀀃􀁗􀁋􀁈􀀃􀃀􀁕􀁖􀁗􀀃􀁇􀁒􀁆􀁘􀁐􀁈􀁑􀁗􀀃􀁌􀁑􀀃􀁗􀁋􀁈􀀃􀁕􀁈􀁖􀁋􀁘􀁉􀃁􀁈􀁇􀀃􀁐􀁒􀁙􀁌􀁈􀀃􀁕􀁈􀁙􀁌􀁈􀁚􀀃􀁇􀁄􀁗􀁄􀁖􀁈􀁗􀀝

### We will use Python's regular expression (regex) library, re, as shown here:

In [19]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emotions = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emotions).replace('-', ''))
    return text



In [20]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

Lastly, since we will make use of the cleaned text data over and over again during the
next sections, let us now apply our preprocessor function to all the movie reviews
in our DataFrame:

In [21]:
df['reviews'] = df['review'].apply(preprocessor)

In [22]:
df['reviews']

0        in 1974 the teenager martha moxley maggie grac...
1        ok so i really like kris kristofferson and his...
2         spoiler do not read this if you think about w...
3        hi for all the people who have seen this wonde...
4        i recently bought the dvd forgetting just how ...
                               ...                        
49995    ok lets start with the best the building altho...
49996    the british heritage film industry is out of c...
49997    i don t even know where to begin on this one i...
49998    richard tyler is a little boy who is scared of...
49999    i waited long to watch this movie also because...
Name: reviews, Length: 50000, dtype: object

### Processing documents into tokens

After successfully preparing the movie review dataset, we now need to think
about how to split the text corpora into individual elements. One way to tokenize 
documents is to split them into individual words by splitting the cleaned documents
at its whitespace characters:

In [23]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [31]:
# pip install nltk

In [24]:
from nltk.stem.porter import PorterStemmer

In [25]:
porter = PorterStemmer()

def tokenizers_porter(text):
    return [porter.stem(word) for word in text.split()]

In [27]:
tokenizers_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

#### stop-word removal.

In order to remove stop-words from the movie reviews, we will use the set of 127
English stop-words that is available from the NLTK library, which can be obtained
by calling the nltk.download function

In [28]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Nazbeen-
[nltk_data]     Ai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

[w for w in tokenizer('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'likes', 'running', 'runs', 'lot']

#### Training a logistic regression model for document classificatio

In [30]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

Next, we will use a GridSearchCV

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer': [tokenizer, tokenizers_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizers_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty':['l1', 'l2'],
               'clf__C':[1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)



Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

In [ ]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

In [ ]:
clf = gs_lr_tfidf.best_estimator_

print('Test Accuracy: %.3f'% clf.score(X_test, y_test))

#### Working with bigger data - online  algorithms and out-of-core learning

In [1]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
    + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [2]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # Skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [3]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

We will now define function, get_minibatch, that will take a document stream
from the stream_docs function and return a particular number of documents
specified by the parameter

In [5]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y



In [8]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log_loss', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [9]:
import pyprind

pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:40


In [10]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [13]:
clf = clf.partial_fit(X_test, y_test)

#### Topic modeling with Latent Dirichlet Allocation

In this section, we will introduce a popular technique for topic modeling called
Latent Dirichlet Allocation (LDA).

LDA with scikit-learn

First, we are going to load the dataset into a pandas DataFrame using the local
movie_data.

In [16]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(
    stop_words='english',
    max_df=.1,
    max_features=5000
)

X = count.fit_transform(df['review'].values)


In [18]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
    n_components=10,
    random_state=123,
    learning_method='batch'
)

X_topics = lda.fit_transform(X)

In [19]:
lda.components_.shape

(10, 5000)

In [22]:
n_top_words = 5
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


### Based on reading the five most important words for each topic, we may guess that  the LDA identified the following topics:

1. Generally bad movies (not really a topic category)
2. Movies about families
3. War movies
4. Art movies
5. Crime movies
6. Horror movies
7. Comedy movies
8. Movies somehow related to TV shows
9. Movies based on books
10. Action movies

In [23]:
import os
import pickle

# Ensure the directory for saving objects exists
os.makedirs('pkl_objects', exist_ok=True)

# Save the classifier
with open(os.path.join('pkl_objects', 'classifier.pkl'), 'wb') as model_file:
    pickle.dump(clf, model_file)

# Save the vectorizer
with open(os.path.join('pkl_objects', 'vectorizer.pkl'), 'wb') as vect_file:
    pickle.dump(vect, vect_file)